<a href="https://colab.research.google.com/github/music-ai-644/AI_Study_2022/blob/main/Chapter6_%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로지스틱 회귀 + 데이터 전처리 + 미니 배치 

## 문제 해결을 위한 데이터셋 준비

1. 필요한 패키지, 데이터셋 불러오기


In [33]:
import random
import torch 
import torch.nn as nn 
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()

x = cancer.data
y = cancer.target 

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)  # 수정 금지 
x_train, x_eval, y_train, y_eval = train_test_split(x_train, y_train, stratify=y_train, test_size=0.125, random_state=999)

# train에 대해서 평균과 표준편차를 구한다. 
mean_x = np.mean(x_train, 0)
mean_x = np.expand_dims(mean_x, 0)

std_x = np.std(x_train, 0)
std_x = np.expand_dims(std_x, 0) 

x_train_1 = (x_train - mean_x) / std_x
x_test_1 = (x_test - mean_x) / std_x
x_eval_1 = (x_eval - mean_x) / std_x

# train에 대해서 최댓값과 최솟값을 구한다. 
max_x = np.max(x_train, 0)
min_x = np.min(x_train, 0)

x_train_2 = (x_train - min_x) / (max_x - min_x)
x_test_2 = (x_test - min_x) / (max_x - min_x)
x_eval_2 = (x_eval - min_x) / (max_x - min_x)

In [34]:
x_train_torch = torch.FloatTensor(x_train)
y_train_torch = torch.FloatTensor(y_train)
x_eval_torch = torch.FloatTensor(x_eval)
y_eval_torch = torch.FloatTensor(y_eval) 
x_test_torch = torch.FloatTensor(x_test)
y_test_torch = torch.FloatTensor(y_test)

x_train_1_torch = torch.FloatTensor(x_train_1)
x_test_1_torch = torch.FloatTensor(x_test_1)
x_eval_1_torch = torch.FloatTensor(x_eval_1)

x_train_2_torch = torch.FloatTensor(x_train_2)
x_test_2_torch = torch.FloatTensor(x_test_2)
x_eval_2_torch = torch.FloatTensor(x_eval_2)


2. 미니 배치 준비 
- np.random.permutation 
- data indexing 

3. 직접 모델 만들어보기 

In [35]:
# Step 1 모델 생성 
model = nn.Sequential(
        nn.Linear(30, 100),
        nn.ReLU(), 
        nn.Linear(100, 1),
        nn.Sigmoid()
        )

In [36]:
# Step 2 업데이트 툴 설정 
optim = torch.optim.Adam(model.parameters(), lr=1e-4)

In [37]:
# Step 3 오차 함수 설정 
criterion = nn.BCELoss() 

In [38]:
# Step 4 오차 계산 함수 설정
def cal_loss(x, y):
  loss = criterion(x, y)
  return loss 

In [39]:
# Step 5 학습 
epochs = 10
for i in range(epochs):
    for x_i, y_i in zip(x_train_torch, y_train_torch):
      pred_y = model(x_i)
      loss = cal_loss(pred_y[0], y_i)
      model.zero_grad() 
      optim.zero_grad()
      loss.backward()
      optim.step()
    
    print("epoch {} : err = {:.4f}".format(i + 1, loss))

epoch 1 : err = 0.0511
epoch 2 : err = 0.0317
epoch 3 : err = 0.0172
epoch 4 : err = 0.0135
epoch 5 : err = 0.0117
epoch 6 : err = 0.0105
epoch 7 : err = 0.0095
epoch 8 : err = 0.0086
epoch 9 : err = 0.0079
epoch 10 : err = 0.0073


In [43]:
# Step 6 예측
def predict(x):
    pred = [model(x_i).data.numpy()[0] for x_i in x]
    return np.array(pred) > 0.5

In [44]:
result = np.mean(predict(x_test_torch) == y_test)
print("정확도 : {:%}".format(result))

정확도 : 92.982456%
